In [1]:
import sys, os, datetime, dotenv, time, re, json
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from transformers import AutoTokenizer, AutoModelForCausalLM
import string
import nltk
from youtube_transcript_api import YouTubeTranscriptApi


/opt/homebrew/anaconda3/envs/hackathon24s/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
TRANSCRIPT_DIR = "transcripts"
video_links = [
    {"link": "https://www.youtube.com/watch?v=ICnFtfN-sUc", "name": "twitch"},
    {"link": "https://www.youtube.com/watch?v=Yqasp_4x6zw", "name": "mongodb"}
]


In [7]:
def download_transcript(id: str, filename: str, overwrite: bool) -> None:

    if not overwrite and os.path.exists(filename):
        print(f"Transcript {filename} already downloaded")
        return

    transcript = YouTubeTranscriptApi.get_transcript(
        id, languages=('en', 'en-US', 'en-GB'))
    global_time = 0
    print(f"Downloading transcript for {id} to {filename}")
    with open(filename, "w") as f:
        for line in transcript:
            new_line = ""
            # add timestamp
            new_line += f"[[{global_time}]]"
            new_line += line['text']
            f.write(f'{new_line}\n')
            global_time += float(line['duration'])
            global_time = round(global_time, 2)

        print(f"Downloaded transcript for {id} to {filename}")
        return

In [8]:
transcripts = []
for video in video_links:
    id = video['link'].split("v=")[1]
    name = video['name']
    file_path = f"{TRANSCRIPT_DIR}/transcript_{name}.txt"
    download_transcript(id, file_path, overwrite=True)


Downloaded transcript for ICnFtfN-sUc to transcripts/transcript_twitch.txt
Downloaded transcript for Yqasp_4x6zw to transcripts/transcript_mongodb.txt
